# ETL UNICOS desde XLSX Multi-Hoja

Este notebook carga un archivo Excel con múltiples hojas, extrae registros de recurso
(Equipos, Mano de Obra, Materiales, Transporte) clasificados por primera letra,
y genera un DataFrame final UNICOS con deduplicación y normalización.

**Entrada**: Archivo .xlsx con 100+ hojas numéricas (001, 002, etc.) y auxiliares (Aux.01, etc.)

**Salida**: DataFrame UNICOS con columnas normalizadas, sin duplicados, ordenado.


## 📘 Documento de Reglas y Contrato del ETL UNICOS

### 🎯 Objetivo
Unificar recursos desde un Excel multi-hoja en un único dataset **UNICOS**, extrayendo **Equipos**, **Mano de Obra**, **Materiales** y **Transporte** con sus **unidades**, **precio unitario (tarifa)**, **CPC Elemento**, **NP/ND/EP** y **VAE (%)**.

### ✅ Contrato de Entrada (Excel)
**Formato esperado**:
- Archivo `.xlsx` con múltiples hojas.
- Hojas numéricas con formato `###` (por defecto `001`, `002`, etc.).
- Opcional: hojas auxiliares `Aux.*`.

**Columnas requeridas (por letra)**:
- **A → Column1**: Código o identificador (se usa para clasificar recurso).
- **B → Column2**: Código alterno (si existe).
- **C → Column3**: Descripción.
- **E → Column5**: Tarifa (Equipos/Mano de Obra) o Unidad (Materiales/Transporte).
- **G → Column7**: Precio unitario (Materiales/Transporte).
- **J → Column10**: CPC Elemento.
- **K → Column11**: NP/ND/EP.
- **L → Column12**: VAE (%).

**Filas inválidas**:
- Vacías en descripción (Column3).
- Encabezados de sección (EQUIPOS, MANO DE OBRA, MATERIALES, TRANSPORTE).

### ⚙️ Configuración
- `sheet_pattern`: regex para hojas numéricas. Default `^\d{3}$`.
- `max_sheet`: máximo de hojas a procesar (0 = todas).
- `include_aux`: incluir hojas `Aux.*`.
- `include_otro`: incluir recursos sin clasificación (OTRO).
- `CODE_PATTERN`: regex para extraer código si viene embebido en texto.

### 🧠 Reglas de Clasificación (RECURSO)
Se clasifica por la **primera letra** del código detectado:
- `M` → EQUIPO
- `N` → MANO DE OBRA
- `O` → MATERIAL
- `P` → TRANSPORTE
- Otro → OTRO

### 🔎 Reglas de Extracción
- **DESCRIPCION**: Column3 (limpia).
- **PRECIO UNITARIO**:
  - Equipos y Mano de Obra → Column5 (tarifa/costo hora)
  - Materiales y Transporte → Column7 (precio unitario)
- **UNIDAD**:
  - Equipos y Mano de Obra → vacío
  - Materiales y Transporte → Column5
- **CPC ELEMENTO**: Column10
- **NP/ND/EP**: Column11
- **VAE (%)**: Column12 (se mantiene como porcentaje numérico, ej. 40.0)

> Equipos y Mano de Obra tienen además **Rendimiento (h/U)** en el Excel,
> pero **no se exporta** en la salida final a menos que se solicite.

### 🔧 Normalización
- **Números**: acepta formatos `1,234.56`, `1.234,56`, `10%`.
- **Texto**: se limpia con `strip()`.

### 🧩 Agrupación y Unificación (UNICOS)
- Se concatenan todas las hojas válidas.
- Se eliminan duplicados según columnas finales.
- Orden final por **DESCRIPCION**.

### 📤 Salida Final (UNICOS)
**Columnas esperadas** (sin código ni hoja):
- `DESCRIPCION`
- `UNIDAD` (si aplica)
- `PRECIO UNITARIO`
- `CPC ELEMENTO`
- `NP/ND/EP`
- `VAE (%)`
- `RECURSO`

---

### 🗺️ Diagrama del Flujo ETL (Mermaid)

```mermaid
flowchart TD
    A[Inicio] --> B[Cargar archivo Excel]
    B --> C[Seleccionar hojas por patrón ### y Aux.*]
    C --> D[Leer columnas A,B,C,E,G,J,K,L]
    D --> E[Filtrar filas inválidas y encabezados]
    E --> F[Detectar código y clasificar recurso]
    F --> G[Extraer descripción, unidad y precio unitario]
    G --> H[Normalizar texto y números]
    H --> I[Concatenar todas las hojas]
    I --> J[Eliminar duplicados]
    J --> K[Ordenar por descripción]
    K --> L[Exportar UNICOS]
```


In [19]:
# %% Setup de entorno
# Esta celda prepara el entorno del notebook:
# 1) Verifica/instala dependencias
# 2) Detecta librerias opcionales
# 3) Importa todo lo necesario para el ETL

import sys
import os
import re
import time
from pathlib import Path
from typing import Dict, List, Tuple, Optional, Set
import warnings
import subprocess
import importlib.util
warnings.filterwarnings('ignore')

print("\n" + "="*60)
print("SETUP: Instalando dependencias...")
print("="*60)
print(f"Python executable: {sys.executable}")
print(f"Python version: {sys.version.split()[0]}")

# Paquetes requeridos para el flujo principal
required_packages = {
    'pandas': 'pandas',
    'numpy': 'numpy',
    'openpyxl': 'openpyxl',
    'ipywidgets': 'ipywidgets',
}

def install_if_missing(package_name, import_name=None):
    """Instala el paquete solo si no existe en el kernel actual."""
    import_name = import_name or package_name
    if importlib.util.find_spec(import_name) is not None:
        print(f"✓ {package_name} ya está instalado")
        return

    print(f"⏳ Instalando {package_name}...")
    try:
        # Importante: usar sys.executable asegura instalar en el mismo kernel
        result = subprocess.run(
            [sys.executable, "-m", "pip", "install", "-q", package_name],
            capture_output=True,
            text=True,
            check=True,
        )
        if result.stdout.strip():
            print(result.stdout.strip())
        if result.stderr.strip():
            print(result.stderr.strip())
        print(f"✓ {package_name} instalado")
    except subprocess.CalledProcessError as e:
        print(f"✗ Error instalando {package_name}")
        if e.stdout:
            print(e.stdout)
        if e.stderr:
            print(e.stderr)
        raise

    # Verificar import después de instalar
    if importlib.util.find_spec(import_name) is None:
        raise ImportError(f"No se pudo importar {import_name} después de la instalación")

# Instalar/verificar requeridos
for pkg, imp in required_packages.items():
    install_if_missing(pkg, imp)

# Opcional: python-calamine para lectura ultra rápida
print("\n⏳ Verificando python-calamine (lectura ultra-rapida)...")
try:
    import calamine
    CALAMINE_AVAILABLE = True
    print("✓ python-calamine disponible")
except ImportError:
    CALAMINE_AVAILABLE = False
    print("ℹ python-calamine no disponible (usando openpyxl)")
    print("  Para instalar: pip install python-calamine")

# Opcional: itables para visualización interactiva
print("\n⏳ Verificando itables (visor interactivo)...")
try:
    import itables
    ITABLES_AVAILABLE = True
    print("✓ itables disponible")
except ImportError:
    ITABLES_AVAILABLE = False
    print("ℹ itables no disponible")
    print("  Para instalar: pip install itables")

# Importar dependencias luego de instalarlas
import pandas as pd
import numpy as np

import ipywidgets as widgets
from ipywidgets import FileUpload, VBox, HBox, Button, Label, HTML, Output, IntText, Text, Checkbox

print("\n" + "="*60)
print("✓ Entorno configurado correctamente")
print("="*60 + "\n")



SETUP: Instalando dependencias...
Python executable: /home/codevars/Projects/ETL_EXCEL APUS/.venv/bin/python
Python version: 3.12.3
✓ pandas ya está instalado
✓ numpy ya está instalado
✓ openpyxl ya está instalado
✓ ipywidgets ya está instalado

⏳ Verificando python-calamine (lectura ultra-rapida)...
ℹ python-calamine no disponible (usando openpyxl)
  Para instalar: pip install python-calamine

⏳ Verificando itables (visor interactivo)...
ℹ itables no disponible
  Para instalar: pip install itables

✓ Entorno configurado correctamente



In [ ]:
# %% Configuración de carpeta de entrada y carga de archivos

import shutil
from IPython.display import display, clear_output

# Crear carpeta de entrada si no existe
INPUT_FOLDER = Path('./input')
INPUT_FOLDER.mkdir(exist_ok=True)

OUTPUT_FOLDER = Path('./output')
OUTPUT_FOLDER.mkdir(exist_ok=True)

# Diccionario global para almacenar la ruta actual del archivo
current_file_path = {'path': None}

print("="*70)
print("📁 CONFIGURACIÓN DE CARPETAS")
print("="*70)
print(f"Carpeta de entrada: {INPUT_FOLDER.absolute()}")
print(f"Carpeta de salida: {OUTPUT_FOLDER.absolute()}")

# UI para cargar archivo desde carpeta de entrada
input_output = Output()
input_status = Output()

input_file_upload = FileUpload(
    accept='.xlsx',
    multiple=False,
    description='📤 Selecciona tu Excel:',
    layout=widgets.Layout(width='500px')
)

def list_input_files():
    """Lista archivos Excel en la carpeta de entrada."""
    excel_files = list(INPUT_FOLDER.glob('*.xlsx')) + list(INPUT_FOLDER.glob('*.xls'))
    return excel_files

def handle_input_upload(change):
    """Guarda el archivo en la carpeta de entrada."""
    input_status.clear_output()
    if input_file_upload.value:
        # ipywidgets >= 8.0 devuelve una tupla de diccionarios
        uploaded_files = input_file_upload.value
        if isinstance(uploaded_files, tuple):
            for file_info in uploaded_files:
                filename = file_info.get('name', 'archivo.xlsx')
                content = file_info.get('content', b'')
                input_path = INPUT_FOLDER / filename
                with open(input_path, 'wb') as f:
                    f.write(content)
                current_file_path['path'] = str(input_path)
                with input_status:
                    print(f"✅ Archivo cargado: {filename}")
                    print(f"📍 Ubicación: {input_path.absolute()}")
        else:
            # Versiones anteriores de ipywidgets
            for filename, filedata in uploaded_files.items():
                input_path = INPUT_FOLDER / filename
                with open(input_path, 'wb') as f:
                    f.write(filedata['content'])
                current_file_path['path'] = str(input_path)
                with input_status:
                    print(f"✅ Archivo cargado: {filename}")
                    print(f"📍 Ubicación: {input_path.absolute()}")

input_file_upload.observe(handle_input_upload, names='value')

# Botón para limpiar carpeta
clean_button = Button(
    description='🗑️ Limpiar entrada',
    button_style='warning',
    layout=widgets.Layout(width='150px')
)

def on_clean_click(b):
    """Limpia la carpeta de entrada."""
    with input_status:
        input_status.clear_output()
        try:
            if list(INPUT_FOLDER.glob('*')):
                shutil.rmtree(INPUT_FOLDER)
                INPUT_FOLDER.mkdir(exist_ok=True)
                current_file_path['path'] = None
                print("✅ Carpeta de entrada limpiada")
            else:
                print("ℹ️ La carpeta ya está vacía")
        except Exception as e:
            print(f"❌ Error al limpiar: {e}")

clean_button.on_click(on_clean_click)

# Botón para listar archivos
list_button = Button(
    description='📋 Listar archivos',
    button_style='info',
    layout=widgets.Layout(width='150px')
)

def on_list_click(b):
    """Lista los archivos en la carpeta de entrada."""
    with input_status:
        input_status.clear_output()
        files = list_input_files()
        if files:
            print("📂 Archivos en la carpeta de entrada:\n")
            for i, f in enumerate(files, 1):
                size_mb = f.stat().st_size / (1024*1024)
                selected = " ← SELECCIONADO" if current_file_path['path'] == str(f) else ""
                print(f"  {i}. {f.name} ({size_mb:.2f} MB){selected}")
        else:
            print("📭 No hay archivos Excel en la carpeta de entrada")

list_button.on_click(on_list_click)

# Construir panel de carga
input_ui = VBox([
    HTML('<h3>📥 Carga de Archivo</h3>'),
    input_file_upload,
    HBox([clean_button, list_button]),
    input_status
])

print("\n")
display(input_ui)

print("\n" + "="*70)
print("✅ Carpeta de entrada configurada correctamente")
print("="*70 + "\n")


📁 CONFIGURACIÓN DE CARPETAS
Carpeta de entrada: /home/codevars/Projects/ETL_EXCEL APUS/input
Carpeta de salida: /home/codevars/Projects/ETL_EXCEL APUS/output





✅ Carpeta de entrada configurada correctamente



In [21]:
# %% Configuración global
# Mapeo de columnas del Excel hacia nombres internos
# Nota: solo se leen columnas específicas para optimizar rendimiento.
COLUMN_MAPPING = {
    'A': 'Column1',
    'B': 'Column2',
    'C': 'Column3',
    'E': 'Column5',
    'G': 'Column7',
    'J': 'Column10',
    'K': 'Column11',
    'L': 'Column12',
}

# Clasificación por primera letra
RESOURCE_TYPE_MAP = {
    'M': 'EQUIPO',
    'N': 'MANO DE OBRA',
    'O': 'MATERIAL',
    'P': 'TRANSPORTE',
}

# Regex para detectar códigos embebidos (ej: M.01, N02, O-03)
CODE_PATTERN = r'([MNOP])\s*[\.-]?\s*\d+'

# Patrones de encabezados que deben excluirse
SECTION_HEADERS_PATTERNS = [
    r'^EQUIPOS',
    r'^MANO\s+DE\s+OBRA',
    r'^MATERIALES?',
    r'^TRANSPORTE',
]

def is_section_header(text: str) -> bool:
    """Detecta si un texto es un encabezado de sección."""
    if pd.isna(text) or text == '':
        return False
    text_upper = str(text).strip().upper()
    for pattern in SECTION_HEADERS_PATTERNS:
        if re.match(pattern, text_upper):
            return True
    return False

print("✓ Configuración global cargada")


✓ Configuración global cargada


In [22]:
# %% Exportación
def export_results():
    """Exporta UNICOS a CSV y XLSX."""
    global df_unicos

    if 'df_unicos' not in globals() or df_unicos is None or df_unicos.empty:
        print("⚠ No hay resultados para exportar. Ejecuta primero el ETL.")
        return

    csv_path = csv_path_text.value
    xlsx_path = xlsx_path_text.value

    try:
        # Exportar CSV
        df_unicos.to_csv(csv_path, index=False, encoding='utf-8-sig')
        print(f"✓ CSV exportado: {csv_path}")

        # Exportar XLSX
        with pd.ExcelWriter(xlsx_path, engine='openpyxl') as writer:
            df_unicos.to_excel(writer, sheet_name='UNICOS', index=False)
        print(f"✓ XLSX exportado: {xlsx_path}")

        print(f"\n📦 Archivos de salida:")
        print(f"   CSV:  {os.path.abspath(csv_path)}")
        print(f"   XLSX: {os.path.abspath(xlsx_path)}")

    except Exception as e:
        print(f"❌ Error en exportación: {e}")

print("✓ export_results() disponible")


✓ export_results() disponible


In [23]:
# %% (Opcional) Carga directa desde ruta
# Solo usa esto si prefieres especificar una ruta en lugar del widget
# Deja vacío para usar el widget de carga

xlsx_path_manual = ""  # Ejemplo: "input/mi_archivo.xlsx"

if xlsx_path_manual:
    path_obj = Path(xlsx_path_manual)
    if path_obj.exists():
        current_file_path['path'] = str(path_obj.absolute())
        print(f"✓ Ruta configurada: {path_obj.absolute()}")
    else:
        print(f"⚠️ Archivo no encontrado: {xlsx_path_manual}")
else:
    print("ℹ️ Usando widget de carga (celda anterior)")


ℹ️ Usando widget de carga (celda anterior)


In [24]:
# %% Funciones ETL - Selección de hojas
def pick_sheets(
    xlsx_path: str,
    sheet_pattern: str = r'^\d{3}$',
    max_sheet: int = None,
    include_aux: bool = True
) -> List[str]:
    """
    Selecciona hojas según patrón regex y límite.
    Respeta el orden original del archivo.
    """
    try:
        excel_file = pd.ExcelFile(xlsx_path)
        all_sheets = excel_file.sheet_names
    except Exception as e:
        print(f"❌ Error abriendo {xlsx_path}: {e}")
        return []

    # Compilar regex del patrón de hojas
    try:
        pattern = re.compile(sheet_pattern)
    except Exception:
        print(f"❌ Patrón regex inválido: {sheet_pattern}")
        return []

    # Hojas numéricas (según patrón)
    numeric_sheets = [s for s in all_sheets if pattern.match(s)]

    # Hojas auxiliares
    aux_sheets = [s for s in all_sheets if s.startswith('Aux.')] if include_aux else []

    # Combinar respetando orden original del archivo
    selected = [s for s in all_sheets if s in numeric_sheets or s in aux_sheets]

    # Limitar cantidad de hojas si se pide
    if max_sheet and max_sheet > 0:
        selected = selected[:max_sheet]

    return selected

print("✓ pick_sheets() cargado")


✓ pick_sheets() cargado


In [25]:
# %% Funciones ETL - Lectura eficiente
def read_sheet_fast(
    xlsx_path: str,
    sheet_name: str,
    column_mapping: Dict[str, str] = COLUMN_MAPPING
) -> Optional[pd.DataFrame]:
    """
    Lee solo columnas necesarias de una hoja.
    Intenta calamine primero (rápido), fallback a openpyxl.
    """
    try:
        # Intentar con python-calamine si está disponible
        if CALAMINE_AVAILABLE:
            try:
                from calamine import load_workbook
                wb = load_workbook(xlsx_path)
                ws = wb.sheet_by_name(sheet_name)

                # Indices 0-based para columnas A, B, C, E, G, J, K, L
                col_indices = {'A': 0, 'B': 1, 'C': 2, 'E': 4, 'G': 6, 'J': 9, 'K': 10, 'L': 11}
                data = {}

                for col_letter, col_name in column_mapping.items():
                    col_idx = col_indices[col_letter]
                    col_data = []
                    for row in ws.rows():
                        if col_idx < len(row):
                            cell = row[col_idx]
                            col_data.append(cell.value if hasattr(cell, 'value') else cell)
                        else:
                            col_data.append(None)
                    data[col_name] = col_data

                df = pd.DataFrame(data)
                return df
            except Exception:
                # Si falla calamine, se intenta openpyxl
                pass

        # Fallback: openpyxl
        from openpyxl import load_workbook
        wb = load_workbook(xlsx_path, data_only=True, read_only=True)
        ws = wb[sheet_name]

        # Construir columnas vacías
        col_letters = list(column_mapping.keys())
        data = {column_mapping[col]: [] for col in col_letters}

        # Recorrer filas y extraer columnas clave
        for row in ws.iter_rows(min_col=1, max_col=12, values_only=False):
            for col_letter, col_name in column_mapping.items():
                col_idx = ord(col_letter) - ord('A')
                if col_idx < len(row):
                    cell = row[col_idx]
                    value = cell.value if hasattr(cell, 'value') else cell
                    data[col_name].append(value)
                else:
                    data[col_name].append(None)

        wb.close()
        df = pd.DataFrame(data)
        return df

    except Exception:
        # En caso de fallo, devolvemos None para que el flujo continúe
        return None

print("✓ read_sheet_fast() cargado")


✓ read_sheet_fast() cargado


In [26]:
# %% Funciones ETL - Filtrado y clasificación
def filter_rows(df: pd.DataFrame) -> pd.DataFrame:
    """
    Excluye:
    - Filas con descripción vacía (Column3)
    - Encabezados de sección
    """
    # Filtrar nulos/vacíos en descripción
    df = df[df['Column3'].notna() & (df['Column3'] != '')]

    # Excluir encabezados de sección (por Column1 o Column3)
    df = df[~df['Column1'].astype(str).apply(is_section_header)]
    df = df[~df['Column3'].astype(str).apply(is_section_header)]

    return df.reset_index(drop=True)

def extract_code(row_dict: Dict) -> str:
    """
    Intenta extraer el código desde Column1/Column2/Column3.
    Si no encuentra patrón, devuelve texto normalizado del campo más confiable.
    """
    candidates = [row_dict.get('Column1', ''), row_dict.get('Column2', ''), row_dict.get('Column3', '')]
    for val in candidates:
        if pd.isna(val) or val == '':
            continue
        m = re.search(CODE_PATTERN, str(val).upper())
        if m:
            return m.group(0).replace(' ', '')

    # Fallback: usar Column1/Column2 sin patrón
    fallback = normalize_text_field(row_dict.get('Column1', '')) or normalize_text_field(row_dict.get('Column2', ''))
    return fallback

def classify_resource(code_or_text: str) -> str:
    """Clasifica recurso por primera letra encontrada."""
    text = str(code_or_text).upper() if code_or_text else ''
    if text:
        letter = text[0]
        return RESOURCE_TYPE_MAP.get(letter, 'OTRO')
    return 'OTRO'

def normalize_numeric(value, is_percentage=False) -> float:
    """
    Normaliza valores numéricos con múltiples formatos:
    - "1,234.56", "1.234,56", "1234,56", "1234.56", "10%", "0.1"

    Nota: si is_percentage=True, convierte 40 -> 0.40
    """
    if pd.isna(value) or value == '':
        return 0.0

    value_str = str(value).strip()

    # Quitar símbolo % si existe (pero no convertir automáticamente)
    if '%' in value_str:
        value_str = value_str.replace('%', '').strip()

    # Detectar separador decimal (coma o punto)
    comma_count = value_str.count(',')
    dot_count = value_str.count('.')

    try:
        if comma_count > 1 or dot_count > 1:
            if ',' in value_str and '.' in value_str:
                # Decide si el último separador es decimal
                if value_str.rindex(',') > value_str.rindex('.'):
                    # Formato: 1.234,56
                    value_str = value_str.replace('.', '').replace(',', '.')
                else:
                    # Formato: 1,234.56
                    value_str = value_str.replace(',', '')
            elif comma_count > 1:
                # Múltiples comas: las primeras son miles
                value_str = value_str.replace(',', '.', comma_count - 1).replace(',', '.')
        elif comma_count == 1 and dot_count == 0:
            # Podría ser 1,5 o 1.500 dependiendo del locale
            parts = value_str.split(',')
            if len(parts[1]) > 2:
                value_str = value_str.replace(',', '')
            else:
                value_str = value_str.replace(',', '.')

        result = float(value_str)

        # Si se pide convertir porcentaje a fracción
        if is_percentage:
            result = result / 100.0

        return result
    except Exception:
        return 0.0

def normalize_text_field(value) -> str:
    """Normaliza campos de texto."""
    if pd.isna(value) or value == '':
        return ''
    return str(value).strip()

print("✓ Funciones de filtrado cargadas")


✓ Funciones de filtrado cargadas


In [27]:
# %% Funciones ETL - Procesamiento de filas
def process_row(row_dict: Dict) -> Optional[Dict]:
    """Procesa una fila y devuelve dict normalizado o None si no es válida."""
    try:
        # Extraer columnas según el mapeo
        col1 = row_dict.get('Column1', '')
        col2 = row_dict.get('Column2', '')
        col3 = row_dict.get('Column3', '')
        col5 = row_dict.get('Column5', '')
        col7 = row_dict.get('Column7', '')
        col10 = row_dict.get('Column10', '')
        col11 = row_dict.get('Column11', '')
        col12 = row_dict.get('Column12', '')

        # Validación: debe haber descripción
        if pd.isna(col3) or col3 == '':
            return None

        # Excluir encabezados
        if is_section_header(col1) or is_section_header(col3):
            return None

        # Detectar código y clasificar recurso
        codigo = extract_code(row_dict)
        recurso = classify_resource(codigo)

        # DESCRIPCION
        descripcion = normalize_text_field(col3)

        # PRECIO UNITARIO y UNIDAD según RECURSO
        if recurso in ('EQUIPO', 'MANO DE OBRA'):
            # Tarifa/costo hora
            precio_unitario = normalize_numeric(col5)
            unidad = ''
        else:  # MATERIAL, TRANSPORTE, OTRO
            precio_unitario = normalize_numeric(col7)
            unidad = normalize_text_field(col5)

        # Otros campos
        cpc = normalize_text_field(col10)
        np_nd = normalize_text_field(col11)
        vae = normalize_numeric(col12, is_percentage=False)

        result = {
            'DESCRIPCION': descripcion,
            'UNIDAD': unidad,
            'PRECIO UNITARIO': precio_unitario,
            'CPC ELEMENTO': cpc,
            'NP/ND/EP': np_nd if np_nd else '',
            'VAE (%)': vae,
            'RECURSO': recurso
        }

        return result
    except Exception:
        # Si algo falla en una fila, no detener todo el ETL
        return None

print("✓ process_row() cargado")


✓ process_row() cargado


In [28]:
# %% Funciones ETL - Construcción del dataset UNICOS
def build_unicos(
    all_rows: List[Dict],
    include_otro: bool = False
) -> pd.DataFrame:
    """
    Construye DataFrame UNICOS final:
    - Concatena todas las filas
    - Deduplicación
    - Normalización
    - Ordenamiento
    """
    if not all_rows:
        print("❌ No hay filas para procesar")
        return pd.DataFrame()

    df = pd.DataFrame(all_rows)

    # Excluir OTRO si aplica
    if not include_otro:
        df = df[df['RECURSO'] != 'OTRO']

    # Columnas finales (sin CODIGO ni HOJA)
    final_cols = ['DESCRIPCION', 'UNIDAD', 'PRECIO UNITARIO', 'CPC ELEMENTO', 'NP/ND/EP', 'VAE (%)', 'RECURSO']
    df = df[final_cols]

    # Deduplicación exacta de filas
    df = df.drop_duplicates()

    # Normalización post-dedup
    df['NP/ND/EP'] = df['NP/ND/EP'].fillna('').astype(str)
    df['VAE (%)'] = df['VAE (%)'].fillna(0).astype(float)
    df['PRECIO UNITARIO'] = df['PRECIO UNITARIO'].fillna(0).astype(float)
    df['UNIDAD'] = df['UNIDAD'].fillna('').astype(str)

    # Ordenar por DESCRIPCION
    df = df.sort_values('DESCRIPCION', ignore_index=True)

    return df

print("✓ build_unicos() cargado")


✓ build_unicos() cargado


In [29]:
# %% Ejecución del ETL
def run_etl(
    xlsx_path: str,
    max_sheet: int = None,
    include_aux: bool = True,
    include_otro: bool = False,
    sheet_pattern: str = r'^\d{3}$'
) -> Tuple[pd.DataFrame, Dict]:
    """
    Ejecuta el pipeline ETL completo.
    Devuelve (df_unicos, stats_dict)
    """
    stats = {
        'total_sheets': 0,
        'sheets_ok': 0,
        'sheets_failed': 0,
        'total_rows_extracted': 0,
        'duplicates_removed': 0,
        'final_rows': 0,
        'time_seconds': 0
    }

    start_time = time.time()

    # 1. Seleccionar hojas
    sheets_to_process = pick_sheets(
        xlsx_path,
        sheet_pattern=sheet_pattern,
        max_sheet=max_sheet,
        include_aux=include_aux
    )

    stats['total_sheets'] = len(sheets_to_process)
    print(f"\n📊 Hojas a procesar: {stats['total_sheets']}")
    print(f"   Patrón: {sheet_pattern}")
    print(f"   Include Aux: {include_aux}")
    print(f"   Max: {max_sheet if max_sheet and max_sheet > 0 else 'todas'}\n")

    all_rows = []

    # 2. Procesar cada hoja
    for i, sheet_name in enumerate(sheets_to_process, 1):
        print(f"[{i:3d}/{stats['total_sheets']}] Leyendo '{sheet_name}'...", end=' ')

        df = read_sheet_fast(xlsx_path, sheet_name)

        if df is None or df.empty:
            print("⚠ Vacía o error")
            stats['sheets_failed'] += 1
            continue

        # Filtrar filas válidas
        df = filter_rows(df)

        if df.empty:
            print("⚠ Sin filas válidas")
            stats['sheets_failed'] += 1
            continue

        # Procesar cada fila
        sheet_rows = 0
        for _, row in df.iterrows():
            processed = process_row(row.to_dict())
            if processed:
                all_rows.append(processed)
                sheet_rows += 1

        stats['sheets_ok'] += 1
        stats['total_rows_extracted'] += sheet_rows
        print(f"✓ {sheet_rows} filas")

    # 3. Construir UNICOS
    print(f"\n🔄 Deduplicando {stats['total_rows_extracted']} filas...")
    df_unicos = build_unicos(all_rows, include_otro=include_otro)

    stats['final_rows'] = len(df_unicos)
    stats['duplicates_removed'] = stats['total_rows_extracted'] - stats['final_rows']
    stats['time_seconds'] = time.time() - start_time

    print(f"\n📈 Resultados:")
    print(f"   Hojas procesadas: {stats['sheets_ok']}/{stats['total_sheets']}")
    print(f"   Filas extraídas: {stats['total_rows_extracted']}")
    print(f"   Duplicados removidos: {stats['duplicates_removed']}")
    print(f"   Filas finales (UNICOS): {stats['final_rows']}")
    print(f"   Tiempo: {stats['time_seconds']:.2f}s")

    return df_unicos, stats

print("✓ run_etl() cargado")


✓ run_etl() cargado


In [30]:
# %% Callback para ejecutar ETL
def on_execute_click(b):
    # Limpiar salida previa
    output_status.clear_output()

    # Determinar ruta
    xlsx_path = manual_path_text.value.strip() if manual_path_text.value.strip() else current_file_path['path']

    if not xlsx_path:
        with output_status:
            print("❌ Por favor sube un archivo o ingresa una ruta manual")
        return

    if not os.path.exists(xlsx_path):
        with output_status:
            print(f"❌ Archivo no encontrado: {xlsx_path}")
        return

    with output_status:
        try:
            # Parámetros desde UI
            max_sheet = max_sheet_input.value if max_sheet_input.value > 0 else None
            include_aux = include_aux_checkbox.value
            include_otro = include_otro_checkbox.value
            sheet_pattern = sheet_pattern_text.value
            csv_path = csv_path_text.value
            xlsx_path_export = xlsx_path_text.value

            # Ejecutar ETL
            df_unicos, stats = run_etl(
                xlsx_path,
                max_sheet=max_sheet,
                include_aux=include_aux,
                include_otro=include_otro,
                sheet_pattern=sheet_pattern
            )

            # Guardar resultado en variables globales
            globals()['df_unicos'] = df_unicos
            globals()['etl_stats'] = stats

            print("\n✓ ETL ejecutado exitosamente")

            # Vista previa
            print("\n📋 Primeras 10 filas:")
            cols_preview = ['DESCRIPCION', 'UNIDAD', 'PRECIO UNITARIO', 'CPC ELEMENTO', 'NP/ND/EP', 'VAE (%)', 'RECURSO']
            print(df_unicos[cols_preview].head(10).to_string(index=False))

            # Distribución por recurso
            print("\n📊 Distribución por RECURSO:")
            print(df_unicos['RECURSO'].value_counts().to_string())

        except Exception as e:
            print(f"❌ Error en ETL: {e}")
            import traceback
            traceback.print_exc()

# Enlazar botón con el callback
execute_button.on_click(on_execute_click)

print("✓ Callback ejecutar ETL asignado")


✓ Callback ejecutar ETL asignado


In [34]:
# %% ⚡ VERIFICACIÓN Y EJECUCIÓN DIRECTA
# Esta celda te permite ejecutar el ETL SIN usar el botón
# Útil para verificar que todo funciona

print("="*70)
print("🔍 VERIFICACIÓN DEL SISTEMA")
print("="*70)

# 1. Verificar archivo cargado
print("\n1️⃣ ARCHIVO CARGADO:")
if current_file_path.get('path'):
    archivo = current_file_path['path']
    if os.path.exists(archivo):
        size_mb = os.path.getsize(archivo) / (1024*1024)
        print(f"   ✅ {archivo}")
        print(f"   📦 Tamaño: {size_mb:.2f} MB")
        
        # Mostrar hojas del archivo
        try:
            excel = pd.ExcelFile(archivo)
            print(f"   📋 Total hojas: {len(excel.sheet_names)}")
            print(f"   📋 Primeras 5: {excel.sheet_names[:5]}")
        except Exception as e:
            print(f"   ⚠️ Error leyendo hojas: {e}")
    else:
        print(f"   ❌ Archivo no existe: {archivo}")
else:
    print("   ❌ No hay archivo cargado")
    print("   👆 Sube un archivo en la celda 2 (widget de carga)")

# 2. Verificar funciones cargadas
print("\n2️⃣ FUNCIONES CARGADAS:")
funciones = ['pick_sheets', 'read_sheet_fast', 'filter_rows', 'run_etl', 'build_unicos']
for f in funciones:
    if f in dir():
        print(f"   ✅ {f}()")
    else:
        print(f"   ❌ {f}() - FALTA EJECUTAR CELDAS ANTERIORES")

# 3. Verificar variables globales
print("\n3️⃣ VARIABLES:")
print(f"   INPUT_FOLDER: {INPUT_FOLDER.absolute() if 'INPUT_FOLDER' in dir() else '❌ No definida'}")
print(f"   OUTPUT_FOLDER: {OUTPUT_FOLDER.absolute() if 'OUTPUT_FOLDER' in dir() else '❌ No definida'}")

print("\n" + "="*70)

# EJECUCIÓN DIRECTA (descomenta para ejecutar sin botón)
if current_file_path.get('path') and os.path.exists(current_file_path['path']):
    print("\n🚀 ¿Quieres ejecutar el ETL ahora?")
    print("   Descomenta las líneas de abajo y ejecuta esta celda nuevamente")
    print("""
--- DESCOMENTA ESTO PARA EJECUTAR ---
df_unicos, etl_stats = run_etl(
    current_file_path['path'],
    max_sheet=5,  # Solo 5 hojas para prueba rápida
    include_aux=True,
    include_otro=False,
    sheet_pattern=r'^\\d{3}$'
)
print("\\n✅ ETL completado! Resultado en: df_unicos")
display(df_unicos.head(10))
""")
else:
    print("\n⚠️ Primero carga un archivo Excel para poder ejecutar el ETL")


🔍 VERIFICACIÓN DEL SISTEMA

1️⃣ ARCHIVO CARGADO:
   ✅ input/00 APUS 9.2 E.T. OTRA VERSIÓN.xlsx
   📦 Tamaño: 3.79 MB
   📋 Total hojas: 595
   📋 Primeras 5: ['Datos', 'PRES.', 'UNICOS', 'ÍNDICE', '001']

2️⃣ FUNCIONES CARGADAS:
   ✅ pick_sheets()
   ✅ read_sheet_fast()
   ✅ filter_rows()
   ✅ run_etl()
   ✅ build_unicos()

3️⃣ VARIABLES:
   INPUT_FOLDER: /home/codevars/Projects/ETL_EXCEL APUS/input
   OUTPUT_FOLDER: /home/codevars/Projects/ETL_EXCEL APUS/output


🚀 ¿Quieres ejecutar el ETL ahora?
   Descomenta las líneas de abajo y ejecuta esta celda nuevamente

--- DESCOMENTA ESTO PARA EJECUTAR ---
df_unicos, etl_stats = run_etl(
    current_file_path['path'],
    max_sheet=5,  # Solo 5 hojas para prueba rápida
    include_aux=True,
    include_otro=False,
    sheet_pattern=r'^\d{3}$'
)
print("\n✅ ETL completado! Resultado en: df_unicos")
display(df_unicos.head(10))



In [32]:
# %% Visor interactivo de resultados
def display_results():
    """Muestra resultados del ETL en formato interactivo."""
    global df_unicos

    if 'df_unicos' not in globals() or df_unicos is None or df_unicos.empty:
        print("⚠ Ejecuta primero el ETL (botón verde)")
        return

    print(f"\n{'='*70}")
    print(f"Dataset UNICOS - {len(df_unicos)} filas totales")
    print(f"{'='*70}\n")

    if ITABLES_AVAILABLE:
        print("📊 Usando visor interactivo itables (búsqueda, ordenamiento, paginación)\n")
        itables.show(df_unicos, maxRows=500)
    else:
        print("📊 Visor estándar (primeras 50 filas):\n")
        pd.set_option('display.max_rows', 50)
        pd.set_option('display.max_columns', None)
        pd.set_option('display.width', None)
        display(df_unicos)
        pd.reset_option('display.max_rows')

print("✓ display_results() disponible")


✓ display_results() disponible


In [33]:
# %% Exportación de resultados
def export_results(include_hoja_column: bool = False):
    """
    Exporta los resultados del ETL a CSV y XLSX.
    
    Args:
        include_hoja_column: Si True, incluye la columna HOJA en los archivos
    """
    if 'df_unicos' not in globals() or df_unicos is None or len(df_unicos) == 0:
        print("⚠️ No hay resultados para exportar")
        print("Ejecuta primero el ETL")
        return

    csv_path = Path(csv_path_text.value)
    xlsx_path = Path(xlsx_path_text.value)

    # Asegurar que existan las carpetas
    csv_path.parent.mkdir(parents=True, exist_ok=True)
    xlsx_path.parent.mkdir(parents=True, exist_ok=True)

    # Preparar datos para exportar
    df_export = df_unicos.copy()
    if not include_hoja_column and 'HOJA' in df_export.columns:
        df_export = df_export.drop(columns=['HOJA'])

    try:
        # Exportar CSV
        df_export.to_csv(csv_path, index=False, encoding='utf-8-sig')
        print(f"✅ CSV: {csv_path.absolute()}")

        # Exportar XLSX
        with pd.ExcelWriter(xlsx_path, engine='openpyxl') as writer:
            df_export.to_excel(writer, sheet_name='UNICOS', index=False)
        print(f"✅ XLSX: {xlsx_path.absolute()}")

        print(f"\n📦 Archivos guardados en carpeta: {OUTPUT_FOLDER.absolute()}")
        
        # Mostrar tamaños de archivo
        if csv_path.exists():
            csv_size = csv_path.stat().st_size / 1024
            print(f"   📄 CSV: {csv_size:.2f} KB")
        if xlsx_path.exists():
            xlsx_size = xlsx_path.stat().st_size / 1024
            print(f"   📊 XLSX: {xlsx_size:.2f} KB")

    except Exception as e:
        print(f"❌ Error en exportación: {e}")

print("✓ export_results() disponible")


✓ export_results() disponible


---

## 🎯 Resumen de uso

### Flujo completo:
1. **Ejecuta todas las celdas** desde el principio (Setup → Carga → Configuración → Funciones → Callback)
2. **Sube tu archivo Excel** usando el widget de carga
3. **Configura parámetros** si es necesario
4. **Presiona "▶️ Ejecutar ETL"**
5. **Ve los resultados** con `display_results()`
6. **Exporta** con `export_results()`

### Comandos útiles después del ETL:
```python
# Ver resultados
display_results()

# Exportar a CSV y XLSX
export_results()

# Exportar incluyendo columna HOJA
export_results(include_hoja_column=True)

# Ver estadísticas
print(etl_stats)

# Filtrar solo EQUIPOS
df_equipos = df_unicos[df_unicos['RECURSO'] == 'EQUIPO']
print(f"Total EQUIPOS: {len(df_equipos)}")

# Buscar por descripción
df_search = df_unicos[df_unicos['DESCRIPCION'].str.contains('MOTOR', case=False)]
display(df_search)
```
